## Pong 04

In [1]:
Versión completa del juego. Usamos sprites perp para darle un aspecto más retro aun.

SyntaxError: invalid syntax (<ipython-input-1-4bbfe0d7feb6>, line 1)

In [2]:
version = '4.0'

import math
import pygame
import random
import time

from pygame.locals import Rect
from math import pi
from vectores import Vector2

SIZE = WIDTH, HEIGHT = 800, 600  # Tamaño de pantalla
CENTER = (WIDTH//2, HEIGHT//2)

BLACK = (0, 0, 0)                # Colores
WHITE = (255, 255, 255)
RED = (255, 0, 0)
GREEN = (0, 255, 0)
BLUE = (0, 0, 255)
CYAN = (0, 255, 255)
MAGENTA = (255, 0, 255)
YELLOW = (255, 255, 0)

FPS = 30                         # Velocidad del juego
        
class Ball(pygame.sprite.Sprite):
    
    def __init__(self, x=0, y=0):
        super().__init__() 
        self.frames = [
            pygame.image.load('pong/ball_00.png'),
            pygame.image.load('pong/ball_01.png'),
            pygame.image.load('pong/ball_02.png'),
        ]
        self.num_frames = len(self.frames)
        self.counter = 0
        self.image = self.frames[self.counter]
        self.rect = self.image.get_rect()
        self.rect.center = (x, y)
        self.speed = 10
        self.reset()

    def reset(self):
        self.rect.center = CENTER
        self.orientation = Vector2(1, 0)
        self.orientation.theta = random.uniform(pi/4, -pi/4)
        
    def bounce(self):
        self.orientation.theta = -self.orientation.theta

    def update(self):
        delta =  self.orientation * self.speed
        self.rect.center = Vector2(self.rect.centerx, self.rect.centery) + delta
        if self.rect.top < 0:
            self.rect.top = 0
            self.bounce()
        elif self.rect.bottom > HEIGHT:
            self.rect.bottom = HEIGHT
            self.bounce()
        self.counter = (self.counter + 1) % self.num_frames
        self.image = self.frames[self.counter]

class InputState:
    
    def __init__(self):
        self.left_up = False
        self.left_down = False
        self.right_up = False
        self.right_down = False
        self.exit = False
        
    def update(self):
        for event in pygame.event.get():
            if event.type == pygame.QUIT:
                self.exit = True
            elif event.type == pygame.KEYDOWN:
                if event.key == pygame.K_ESCAPE:
                    self.exit = True
                elif event.key == pygame.K_w:
                    self.left_up = True
                elif event.key == pygame.K_s:
                    self.left_down = True
                elif event.key == pygame.K_UP:
                    self.right_up = True
                elif event.key == pygame.K_DOWN:
                    self.right_down = True
            elif event.type == pygame.KEYUP:
                if event.key == pygame.K_w:
                    self.left_up = False
                elif event.key == pygame.K_s:
                    self.left_down = False
                elif event.key == pygame.K_UP:
                    self.right_up = False
                elif event.key == pygame.K_DOWN:
                    self.right_down = False


class Paddle(pygame.sprite.Sprite):
    
    def __init__(self, x, y):
        super().__init__()
        self.frames = [
            pygame.image.load('pong/paddle_00.png'),
            pygame.image.load('pong/paddle_01.png'),
            pygame.image.load('pong/paddle_02.png'),
        ]
        self.num_frames = len(self.frames)
        self.counter = 0
        self.image = self.frames[self.counter]
        self.rect = self.image.get_rect()
        self.rect.center = (x, y)
        
        
    def update(self):
        self.rect = self.rect.move(0, self.speed)
        self.counter = (self.counter + 1) % self.num_frames
        self.image = self.frames[self.counter]
        
    def go_up(self):
        self.speed = -10

    def go_down(self):
        self.speed = 10
        
    def stop(self):
        self.speed = 0

        
class LeftPad(Paddle):
    
    def update(self, inputs):
        if inputs.left_up:
            self.go_up()
        elif inputs.left_down:
            self.go_down()
        else:  
            self.stop()
        super().update()
        
class RightPad(Paddle):
    
    def update(self, inputs):
        if inputs.right_up:
            self.go_up()
        elif inputs.right_down:
            self.go_down()
        else:
            self.stop()        
        super().update()   

        
def check_impact(paddle, ball):
    if ball.rect.colliderect(paddle.rect):
        ball.orientation.theta = pi - ball.orientation.theta

        
def left_wins(ball):
    return ball.rect.left < 0


def right_wins(ball):
     return ball.rect.right > WIDTH
        

score = {
    'left': 0,
    'right': 0,
    }
    
def draw_text(screen, text, position):
    img = font.render(str(text), True, GREEN)
    rect = img.get_rect()
    rect.centerx = position[0]
    rect.centery = position[1]
    screen.blit(img, rect)
    
def draw_score(screen, score, position):
    img = font.render(str(score), True, GREEN)
    rect = img.get_rect()
    rect.centerx = position[0]
    rect.top = position[1]
    screen.blit(img, rect)
    

def draw_game(screen, background, score, counter, trace_on=False):
    screen.blit(background, (0, 0))
    draw_score(screen, score['left'], (320, 50))
    draw_score(screen, score['right'], (470, 50))
    if counter:
        draw_text(screen, counter, CENTER)
    all_objects.draw(screen)
    if trace_on:
        for item in all_objects.sprites():
            pygame.draw.rect(screen, RED, item.rect, 1)
    pygame.display.update()
    
    
def reset_game():
    global counter, ball
    counter = 3 * FPS
    ball.reset()
    
pygame.init()

try:
    pygame.display.set_caption(version)
    background = pygame.image.load('pong/background.png')
    font = pygame.font.Font('pong/fonts/arcade.ttf', 90)

    screen = pygame.display.set_mode(SIZE, 0, 24)
    limits = screen.get_rect()
    
    # Parte de inicialización del juego
    r_paddle = RightPad(WIDTH-50, HEIGHT//2)
    l_paddle = LeftPad(50, HEIGHT//2)
    ball = Ball(limits.centerx, limits.centery)
    all_objects = pygame.sprite.Group(ball, r_paddle, l_paddle)
    clock = pygame.time.Clock()    
    input_state = InputState()
    counter = 3 * FPS
    while True:
        if counter > 0:
            counter -= 1
        else:
            input_state.update()
            if input_state.exit:
                break
            check_impact(r_paddle, ball)
            check_impact(l_paddle, ball)
            r_paddle.update(input_state)
            l_paddle.update(input_state)
            ball.update()  
            if left_wins(ball):
                score['left'] += 1
                reset_game()
            if right_wins(ball):
                score['right'] += 1
                reset_game()
        draw_game(screen, background, score, counter)
        clock.tick(FPS)
        if score['left'] > 15 or score['right'] > 15:
            break
finally:
    pygame.quit()